In [ ]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import folium
from shapely.geometry import box, Point, Polygon
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import yaml

In [ ]:
# get path to project root directory
project_root = Path.cwd().parents[0]

# build path to yaml config file
config_path = project_root / "configs"/"paths.yaml"

In [ ]:
# load yaml file into python dictionary
with open(config_path) as f:
    paths = yaml.safe_load(f)

paths

In [ ]:
# build data directory using paths from yaml config
data_dir = project_root/paths['data']['processed']

# build path to external data using paths from yaml config
data_external = project_root/paths['data']['external']

# finally build path to our deduped settlements geojson file
settlements_path = data_dir / "UNHCR_poc_boundaries-Uganda_attributed_deduped.geojson"

# build path to our regions geojson file
refugee_regions_path = data_external / "refugeehosting_regions.geojson"

In [ ]:
# build path to output directory for processed hexbin data
output_dir = project_root/paths['data']['processed']

# build path to output directory for maps
maps_dir = project_root/paths['outputs']['dynamic_maps']

In [ ]:
# load settlements into a gdf
settlements = gpd.read_file(settlements_path)
settlements.head(1)

In [ ]:
# load regions into a gdf
refugee_regions = gpd.read_file(refugee_regions_path)
refugee_regions = gpd.GeoDataFrame(refugee_regions[['ADM2_EN','ADM1_EN', 'ADM0_EN','geometry']], geometry = 'geometry').to_crs(32636)
refugee_regions

In [ ]:
# plot nakivale and its bounding box
fig, ax = plt.subplots(figsize=(8, 10))

# plot the Nakivale polygon(s)
refugee_regions.plot(ax=ax, color='lightgreen', edgecolor = 'black',linewidth=1, label='Refugee Regions')

# Plot the bounding box outline
settlements.boundary.plot(ax=ax, edgecolor='red', linewidth=1, label='Refugee Settlements')

# Add labels and styling
ax.set_title("Refugee Settlement Containing Regions", fontsize=14)
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
def make_hex_mesh_over_union(polys_gdf, hex_area, origin=None):
    """
    Build a hex grid covering the union of all polygons,
    then return all hexes whose centroids lie within the union (no clipping).
    """
    # sanity
    if polys_gdf.crs is None or polys_gdf.crs.is_geographic:
        raise ValueError("Polygons must be in a projected CRS in meters (e.g., EPSG:32636).")

    # derive side length from target area
    s = sqrt((2 * hex_area) / (3 * sqrt(3)))
    w = 2 * s
    h = sqrt(3) * s
    dx = 1.5 * s
    dy = h

    # union bounds
    polys_gdf = polys_gdf.buffer(3*s)
    minx, miny, maxx, maxy = polys_gdf.total_bounds

    # optional origin so grids align run-to-run (defaults to lower-left bound)
    if origin is None:
        x0, y0 = minx - w, miny - h
    else:
        x0, y0 = origin

    cols = np.arange(x0, maxx + w, dx)
    rows = np.arange(y0, maxy + h, dy)

    
    union = polys_gdf.unary_union

    hexes = []
    for i, x in enumerate(cols):
        for j, y in enumerate(rows):
            y_offset = y + (dy / 2 if i % 2 else 0)
            c = Point(x, y_offset)
            if union.contains(c):
                vertices = [(x + s * np.cos(np.deg2rad(a)),
                             y_offset + s * np.sin(np.deg2rad(a))) for a in range(0, 360, 60)]
                hexes.append(Polygon(vertices))

    return gpd.GeoDataFrame(geometry=hexes, crs=polys_gdf.crs)


In [ ]:
areas = [250000, 62500, 15625]  # in square meters

hex_250k = make_hex_mesh_over_union(refugee_regions, areas[0])
hex_62k  = make_hex_mesh_over_union(refugee_regions, areas[1])
hex_15k  = make_hex_mesh_over_union(refugee_regions, areas[2])

In [ ]:
# plot nakivale and its bounding box
fig, ax = plt.subplots(figsize=(8, 10))

# plot the Nakivale polygon(s)
refugee_regions.plot(ax=ax, color='lightgreen', edgecolor = 'black',linewidth=1, label='Refugee Regions')

# Plot the bounding box outline
settlements.boundary.plot(ax=ax, edgecolor='red', linewidth=1, label='Refugee Settlements')

# Plot the hexgrid
hex_250k.plot(ax = ax, color = 'white', edgecolor = 'black', alpha = 0.6, linewidth = 0.25, label = 'hexgrid' )

# Add labels and styling
ax.set_title("Refugee Settlement Containing Regions", fontsize=14)
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
def process_hex_mesh(hex_gdf, refugee_regions, settlements, out_path):
    # Assign regions by centroid containment
    hex_centroids = hex_gdf.copy()
    hex_centroids["geometry"] = hex_centroids.geometry.centroid

    hex_with_region = (
        gpd.sjoin(
            hex_centroids.assign(OID=np.arange(len(hex_centroids))),
            refugee_regions,
            how="left",
            predicate="within"
        )
        .merge(hex_gdf.assign(OID=np.arange(len(hex_gdf))), on="OID", suffixes=("_pt", ""))
        .drop(columns=["index_right"])
    )

    # Assign settlements by intersection
    hex_with_settlements = (
        gpd.sjoin(
            hex_with_region.assign(OID=np.arange(len(hex_with_region))),
            settlements[['name', 'geometry']],
            how='left',
            predicate='intersects'
        )
        .drop(columns=['geometry_pt', 'index_right'])
    )

    # Reproject, compute centroids, and add lat/lon
    hex_with_settlements = hex_with_settlements.to_crs(4326)
    hex_with_settlements['lat'] = hex_with_settlements.centroid.y
    hex_with_settlements['lon'] = hex_with_settlements.centroid.x
    hex_with_settlements = hex_with_settlements.to_crs(32636)

    # Rename 'name' column
    hex_with_settlements = hex_with_settlements.rename(columns={'name': 'settlement_name'})

    # Reorder columns (geometry last)
    cols = list(hex_with_settlements.columns)
    cols_reordered = [c for c in cols if c != 'geometry'] + ['geometry']
    hex_with_settlements = hex_with_settlements[cols_reordered]

    # Save to GeoJSON
    hex_with_settlements.to_file(out_path, driver="GeoJSON")
    print(f"Saved: {out_path}")

    return hex_with_settlements


In [ ]:
# Run for all resolutions
hex_meshes = {
    "250k": hex_250k,
    "62k": hex_62k,
    "15k": hex_15k
}
for label, hex_gdf in hex_meshes.items():
    out_file = f"uganda_hexbins_{label}_lcluc_v1.geojson"
    out_path = output_dir/out_file
    process_hex_mesh(hex_gdf, refugee_regions, settlements, out_path)

In [ ]:
hex_250k = gpd.read_file(output_dir/'uganda_hexbins_250k_lcluc_v1.geojson')

In [ ]:
import folium

# Base map centered on Uganda
m = folium.Map(location=[1.3733, 32.2903], zoom_start=8, tiles="CartoDB positron")

# Add settlement outlines
folium.GeoJson(
    refugee_regions,
    name="Refugee Host Regions",
    style_function=lambda x: {
        "color": "green",
        "weight": 1,
        "fillOpacity": 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=[refugee_regions.columns[0]])
).add_to(m)


folium.GeoJson(
    settlements,
    name="Settlements",
    style_function=lambda x: {
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=[settlements.columns[0]])
).add_to(m)



folium.GeoJson(
    hex_250k,
    name=f"hexbins",
    style_function=lambda x: {
        "color":  "#E63946",
        "weight": 1,
        "fillOpacity": 0.1,
    },
    tooltip=folium.GeoJsonTooltip(fields=[hex_250k.columns[0]])
).add_to(m)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Save and show
m.save(maps_dir / "hexbins_settlements_map_v6.html")
m
